# Software Requirements Specification for ChemCode
## Samuel J. Crawford




# Table of Contents
<a id="Sec:ToC"></a>

An outline of all sections included in this SRS is recorded here for easy reference.

- [Table of Contents](#Sec:ToC)
- [Introduction](#Sec:Intro)
 - [Scope of Requirements](#Sec:ReqsScope)

# Introduction
<a id="Sec:Intro"></a>

Chemical equations are common ways of representing chemical reactions. The program documented here is called ChemCode.

The following section provides an overview of the Software Requirements Specification (SRS) for ChemCode. This section explains the purpose of this document, the scope of the requirements, the characteristics of the intended reader, and the organization of the document.

## Scope of Requirements
<a id="Sec:ReqsScope"></a>

The scope of the requirements includes all chemical equations with at most one more compound than element.

